# Past Values and Past Errors: The ARMA Model

* Limitations of AR or MA models 

  * Both AR and MA have residuals that is not white noise. This means neither of them is suitable for fitting price return data. 
  * AR limitation: fail to adjust quickly to unexpected shocks --> need MA aspects to smooth out prediction
  * MA limitation:  need a baseline to perform well --> need to use previous values (AR) 
  * AR + MA: solves the issues each one has individually 

* Definition 

  * Equation ARMA(1) 
    $$
    r_t = c + \varphi_1r_{t-1} + \theta_1\epsilon_{t-1} + \epsilon_t
    $$

    * $r_t$, $r_{t-1}$: The values of "r" in the current period and 1 period ago respectively
    * $\epsilon_t$, $\epsilon_{t-1}$ : Residuals for the current period and the 1 period ago respectively
    * $c$: Baseline constant factor 
    * $\varphi_1$: The part of the value last period is relevant in explaining the current one, $|\varphi_n|<1$ to prevent compounded effects exploding in magnitude
    * $\theta_1$: The part of the value last period is relevant in explaining the current one, $|\theta_n|<1$ to prevent compounded effects exploding in magnitude 

  * ARMA(p, q)

    * p: number of lagged values (AR order)
    * q: number of lagged errors (MA order)
    * p and q don't have to be the same value  
    * Actual - prediction -> how foar off our predictions were -> calibrate expections on the go 